In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
df_train=pd.read_csv("/home/aaron/Projects/Janata Hack/Mobility Analytics/av-mobilityanalytics/train_Wc8LBpr.csv",index_col="Trip_ID")
df_test= pd.read_csv("/home/aaron/Projects/Janata Hack/Mobility Analytics/av-mobilityanalytics/test_VsU9xXK.csv",index_col="Trip_ID")
df_submission=pd.read_csv("/home/aaron/Projects/Janata Hack/Mobility Analytics/av-mobilityanalytics/sample_submission_NoPBkjr.csv")

In [3]:
# We will input mode in type of cab and confidence life_style index  
# and input mean in the rest of the columns


In [4]:
for column in ['Type_of_Cab', 'Confidence_Life_Style_Index']:
    df_train[column].fillna(df_train[column].mode()[0], inplace=True)
    df_test[column].fillna(df_test[column].mode()[0], inplace=True)

In [5]:
for column in ['Customer_Since_Months', 'Life_Style_Index','Var1']:
    df_train[column].fillna(df_train[column].mean(), inplace=True)
    df_test[column].fillna(df_test[column].mean(), inplace=True)

In [6]:


#Feature Engineering

X_train = df_train.drop(columns='Surge_Pricing_Type',axis=1)
Y_train = df_train['Surge_Pricing_Type']
X_test = df_test



In [7]:
X_test['Destination_Type'].value_counts()

A    51413
B    19638
C     4913
D     4497
E     1832
F     1272
G     1024
H      864
I      521
J      471
K      422
L      409
M       67
N       52
Name: Destination_Type, dtype: int64

In [8]:
#Label Encoding Type_of_Cab and Destination_Type

label_columns=['Destination_Type']
label_encoder=LabelEncoder()
for col in label_columns:
    X_train[col] = label_encoder.fit_transform(X_train[col])
    X_test[col] = label_encoder.fit_transform(X_test[col])


In [9]:
# Apply one-hot encoder to each column with categorical data
one_hot_cols=['Type_of_Cab','Confidence_Life_Style_Index','Gender']

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_train_f = pd.DataFrame(OH_encoder.fit_transform(X_train[one_hot_cols]))
X_test_f = pd.DataFrame(OH_encoder.fit_transform(X_test[one_hot_cols]))

# One-hot encoding removed index; put it back
X_train_f.index = X_train.index
X_test_f.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(one_hot_cols, axis=1)
num_X_test = X_test.drop(one_hot_cols, axis=1)

# Add one-hot encoded columns to numerical features
F_X_train = pd.concat([num_X_train, X_train_f], axis=1)
F_X_test = pd.concat([num_X_test, X_test_f], axis=1)

In [12]:
# Scaling the Train and Test feature set 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(F_X_train)
X_test_scaled = scaler.transform(F_X_test)

In [17]:
model.fit(X_train_scaled, Y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.07, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_estimators=225,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [18]:
predict_test = model.predict(X_test_scaled)
df_submission.Surge_Pricing_Type = predict_test
df_submission.to_csv('solution.csv',index = False)


In [16]:
model=XGBClassifier(max_depth =5,num_estimators = 225,learning_rate = 0.07)